### Creazione grafo
nodi totali: 3110737 \
nodi che presentano l'anno: 3110687

In [5]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import re

data = pd.read_csv("imdb-actors-actresses-movies.tsv", sep="\t")
#data = pd.read_csv("prova.tsv", sep="\t")
#print(data)
edges_dict = data.to_dict("index")
#print(edges_dict)
actors = data.get("Actor")
actors.drop_duplicates(inplace = True)
actors.reset_index(drop = True, inplace = True)
id_actor_dict = actors.to_dict()
#print(id_actor_dict)
actor_id_dict = {v: k for k ,v in id_actor_dict.items()}
movies = data.get("Movie")
movies.drop_duplicates(inplace = True)
movies.reset_index(drop = True, inplace = True)
movies.index += len(actors)
id_movie_dict = movies.to_dict()
#print(id_movie_dict)
movie_id_dict = {v: k for k, v in id_movie_dict.items()}

In [6]:
G = nx.Graph()
G.add_nodes_from([(id, {"name": a, "type": "actor"}) for a, id in actor_id_dict.items()])
G.add_nodes_from([(id, {"name": m, "type": "movie", "year": int(re.search("(\(\d\d\d\d\))", m).group()[1:-1])}) for m, id in movie_id_dict.items() if re.search("(\(\d\d\d\d\))", m)!=None])
G.add_edges_from([(actor_id_dict[v["Actor"]], movie_id_dict[v["Movie"]]) for k, v in edges_dict.items() if actor_id_dict[v["Actor"]] in G.nodes and movie_id_dict[v["Movie"]] in G.nodes])


""" print(G.nodes.data())
print(G.edges.data())
nx.draw(G) """

' print(G.nodes.data())\nprint(G.edges.data())\nnx.draw(G) '

### Es 1.E
Vengono creati 2 dizionari, le cui istanze sono: Nome dell'attore-Chiave e Anno film (minimo o massimo)-Valore.
Un ciclo for poi gira su tutti i film che hanno come data di produzione un tempo compreso fra *min_year* e *max_year*, e guarda gli attori che hanno partecipato a tali film. Per tali attori aggiorna il valore nel dizionario se l'anno del film è minore del valore presente nel dizionario o crea la voce se esso non è presente.
Il secondo ciclo for fa la stessa cosa però con i tempi massimi.
Infine il terzo ciclo calcola i tempi di attività nel periodo *min_year* e *max_year* e trova l'attore che ha lavorato di più.

In [ ]:
def esercizio1(max_year):
  actor_year_min = {}
  actor_year_max = {}

  for node in G.nodes().data():
    if (node[1]["type"] == "movie" and node[1]["year"] <= max_year):
        for l in G.neighbors(node[0]):
          if(l not in actor_year_min or node[1]["year"] < actor_year_min.get(l)):
            actor_year_min.update({l : node[1]["year"]})

  for node in G.nodes().data():
      if (node[1]["type"] == "movie" and node[1]["year"] <= max_year):
        for l in G.neighbors(node[0]):
          if(l not in actor_year_max or node[1]["year"] > actor_year_max.get(l)):
            actor_year_max.update({l : node[1]["year"]})

  max_work_period = 0
  actor_max = None
  for a in actor_year_min.items():
    if max_work_period < (actor_year_max.get(a[0]) - actor_year_min.get(a[0])):
      actor_max = a[0]
      max_work_period = actor_year_max.get(a[0]) - actor_year_min.get(a[0])
  
  print("L'attore che ha lavorato di più fino al "+str(max_year)+" è: ")
  print(str(actor_max)+" con un periodo di attività di: "+str(max_work_period)+" anni")
  print()

for x in range (1930, 2030, 10):
  esercizio1(x)

### Es 2.1

In [ ]:
def generateSubGraphGCC(G, xmin, xmax):
    SG = G.subgraph([n for n, a in G.nodes().data() if a["type"] == "actor" or (a["year"] > xmin and  a["year"] < xmax)])
    tempGCC = sorted(nx.connected_components(SG), key=len, reverse=True)[0]
    print("a")
    GCC = SG.subgraph(tempGCC)
    degree_dict = dict(GCC.degree())
    m = max(degree_dict.values())
    start_node = [k for k, v in degree_dict.items() if v == m][0]
    computeDiameter(GCC, start_node)

def computeDiameter(G, u):
    i = lb = nx.eccentricity(G, u)
    ub = 2*lb
    Fi = nx.descendants_at_distance(G, u, i)
    Bi = max([nx.eccentricity(G, v) for v in Fi]) 
    while ub > lb: 
        print("b")
        if max([lb, Bi]) > 2*(i - 1):
            print("diametro: ", max([lb, Bi]))
            return max([lb, Bi])
        else: 
            lb = max([lb, Bi])
            ub = 2*(i - 1) 
        i = i - 1 
        Fi = nx.descendants_at_distance(G, u, i) 
        Bi = max([nx.eccentricity(G, v) for v in Fi])

    print("diametro: ", lb)
    return lb

for x in range (1930, 2030, 10):
    generateSubGraphGCC(G, x, x+10)


### Es 3.I

In [23]:
def maxCollab(G):
    collaborations = {}
    for a in G.nodes().data():
        if a[1]["type"] == "actor":
            c = 0
            for n in G.adj[a[0]]:
                c += (len(G.adj[n])-1)
            collaborations.update({a[0]: c})
    m = max(list(collaborations.values()))
    print(m)
    print([k for k, v in collaborations.items() if v == m][0])
    
maxCollab(G)

40386
Flowers, Bess


### Es 4

In [56]:
A = nx.Graph()
A.add_nodes_from([(a, {"id": id}) for a, id in actor_id_dict.items()])
#A.add_nodes_from([a for a in G.nodes().data() if a[1]["bipartite"] == 0])
#A.add_edges_from([(a1, a2) for a1 in A.nodes() for m in G.adj[a1] for a2 in G.adj[m]])

#print(A.nodes.data())
#print(A.edges.data())
#nx.draw(A)

In [ ]:
for k1, v1 in edges_dict.copy().items():
    for k2, v2 in edges_dict.copy().items():
        if v2["Movie"] == v1["Movie"] and v1["Actor"] != v2["Actor"]:
            A.add_edge(v1["Actor"], v2["Actor"])
            edges_dict.pop(k2)
    edges_dict.pop(k1)

    #A.add_edges_from([(v1["Actor"], v2["Actor"]) for k2, v2 in edges_dict.items() if v1["Actor"] != v2["Actor"] and v2["Movie"] == v1["Movie"]])

print(A.nodes.data())
print(A.edges.data())

In [ ]:
for i in A.nodes:
    for j in A.nodes:
        try:
            if nx.shortest_path_length(G, i, j) == 2:
                A.add_edge(i, j)
        except:
            pass
    #A.add_edges_from([(i, j) for j in A.nodes if nx.shortest_path_length(G, i, j) == 2])

print(A.nodes.data())
print(A.edges.data())

In [7]:
from itertools import combinations

A = nx.Graph()
movie_list = id_movie_dict.keys()
for i in movie_list:
    if G.has_node(i):
        actors = list(G.neighbors(i))
        for j in combinations(actors, 2):
            if A.has_edge(*j):
                A[j[0]][j[1]]['weight'] += 1
            else:
                A.add_edge(j[0],j[1], weight = 1)

#print(A.nodes.data())
#print(A.edges.data())



### celle per prove

In [ ]:
diz = {0: "a 1900 (2000)", 1: "b 1920"}
x = re.search("(\(\d\d\d\d\))", diz[1])
for k, v in diz.items():
    print(k)
    print(v)


In [ ]:
print(id_actor_dict[0])
print(actor_id_dict[id_actor_dict[0]])

### file luca
esegue in circa 1 min

In [65]:
""" G = nx.Graph()
f = open("imdb-actors-actresses-movies.tsv")
i = 0
actors_dict = {}
movies_dict = {}
r_dictionary = {}

for line in f:
    x = re.compile('(.+)\t(.+(?<=\()(\d\d\d\d)(?=[/IXV]*\)).*)')
    matched_line = x.match(line)
    if matched_line:
        actor = matched_line.group(1)
        movie = matched_line.group(2)
        year = int(matched_line.group(3))
        G.add_node(actor, id=i, type='actor')
        G.add_node(movie, id=i + 1, type='movie', year=year)
        G.add_edge(actor, movie)
        actors_dict.update({actor: i})
        movies_dict.update({movie: i + 1})
    i += 2

f.close()
n_actors = 0
n_movie = 0
for node in G.nodes(data=True):
    if node[1]['type'] == 'actor':
        n_actors += 1
    else:
        n_movie += 1
print('actors: ', n_actors)
print('movie: ', n_movie) """

" n_actors = 0\nn_movie = 0\nfor node in G.nodes(data=True):\n    if node[1]['type'] == 'actor':\n        n_actors += 1\n    else:\n        n_movie += 1\nprint('actors: ', n_actors)\nprint('movie: ', n_movie) "